# Homework 8
## HW Group C 4: 
### - Aman Krishna
### - Jingwen Li
### - Sean Lin
### - Yazmin Ramirez Delgado

In [110]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from arch import arch_model
from arch.univariate import GARCH, EWMAVariance 
from sklearn import linear_model
import scipy.stats as stats
from statsmodels.regression.rolling import RollingOLS
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.precision", 4)
sns.set(rc={'figure.figsize':(15, 10)})

In [111]:
def regress(y, X, intercept = True, annual_fac=12):
    if intercept == True:
        X_ = sm.add_constant(X)
        reg = sm.OLS(y, X_).fit()
        reg_df = reg.params.to_frame('Regression Parameters')
        reg_df.loc['r-squared'] = reg.rsquared
        reg_df.loc['const'] *= annual_fac
    else:
        reg = sm.OLS(y, X).fit()
        reg_df = reg.params.to_frame('Regression Parameters')
        reg_df.loc['r-squared'] = reg.rsquared
    
    return reg_df

In [112]:
def risk_stats(data, q=0.05):
    df = data.copy()
    df.index = data.index.date
    report = pd.DataFrame(columns = df.columns)
    
    report.loc['Skewness'] = df.skew()
    report.loc['Excess Kurtosis'] = df.kurtosis()
    report.loc['VaR (negated)'] = -df.quantile(q)
    report.loc['Expected Shortfall (negated)'] = -df[df < df.quantile(q)].mean()
    
    cum_ret = (1 + df).cumprod()
    rolling_max = cum_ret.cummax()
    drawdown = (cum_ret - rolling_max) / rolling_max
    report.loc['Max Drawdown'] = drawdown.min()
    report.loc['MDD Start'] = None
    report.loc['MDD End'] = drawdown.idxmin()
    report.loc['Recovery Date'] = None
    
    for col in df.columns:
        report.loc['MDD Start', col] = (rolling_max.loc[:report.loc['MDD End', col]])[col].idxmax()
        recovery_df = (drawdown.loc[report.loc['MDD End', col]:])[col]
        
        try:
            report.loc['Recovery Date', col] = recovery_df[recovery_df >= 0].index[0]

        except:
            report.loc['Recovery Date', col] = None
            report.loc['Recovery period (days)'] = None
    report.loc['Recovery period (days)'] = (report.loc['Recovery Date'] - report.loc['MDD Start']).dt.days
    return round(report,4)
#risk_stats(df, 0.05)

In [113]:
#Calculate mean, standard deviation and sharpe ratio
def mean_vol_sharpe(df,ann=12):
    mean = df.mean() * ann
    volatility = df.std() * np.sqrt(ann)
    sharpe_ratio = mean/volatility
    return pd.DataFrame({'mean': mean, 'volatility': volatility, 'sharpe_ratio': sharpe_ratio})

In [114]:
ltcm = pd.read_excel('ltcm_exhibits_data.xlsx', sheet_name = 1, skiprows=2).rename(columns={"Unnamed: 0": "Date"}).dropna()
ltcm['Date'] = pd.to_datetime(ltcm['Date']).dt.normalize()
ltcm.set_index('Date',inplace=True)
ltcm.head(10)

,Fund Capital ($billions),Gross Monthly Performancea,Net Monthly Performanceb,Index of Net Performance
Date,,,,
1994-03-01,1.1,-0.011,-0.013,0.99
1994-04-01,1.1,0.014,0.008,1.00
1994-05-01,1.2,0.068,0.053,1.05
1994-06-01,1.2,-0.039,-0.029,1.02
1994-07-01,1.4,0.116,0.084,1.10
1994-08-01,1.5,0.038,0.030,1.14
1994-09-01,1.5,-0.004,-0.003,1.13
1994-10-01,1.5,0.010,0.004,1.14
1994-11-01,1.6,0.077,0.061,1.21


In [115]:
SPY = pd.read_excel('gmo_analysis_data.xlsx',2, index_col=0)
rf = pd.read_excel('gmo_analysis_data.xlsx',3, index_col=0)
SPY['rf'] = rf['US3M']
SPY_ex = SPY.subtract(SPY['rf'], axis=0)['SPY'].to_frame('SPY ex')

SPY_ex.head()

,SPY ex
1993-02-28,0.0082
1993-03-31,0.0199
1993-04-30,-0.0281
1993-05-31,0.0244
1993-06-30,0.0011


# 2 LTCM Risk Decomposition

### On Canvas, find the data file, “ltcm exhibits data.xlsx”. Get the gross and net (total) returns of LTCM from “Exhibit 2”.
### Get the returns on SPY as well as the risk-free rate from the file, “gmo analysis data”.

#### 1. Summary stats.
(a) For both the gross and net series of LTCM excess returns, report the mean, volatility, and Sharpe ratios. (Annualize them.) <br>
(b) Report the skewness, kurtosis, and (historic) VaR(.05). <br>
(c) Comment on how these stats compare to SPY and other assets we have seen. How much do they differ between gross and net?

In [116]:
ltcm.index = SPY_ex.loc['1994-03':'1998-07'].index
ltcm_ex = ltcm.subtract(SPY['rf'], axis = 0)
ltcm_ex.dropna(inplace=True)
ltcm_ex['SPY (excess)'] = SPY_ex['SPY ex']
ltcm_ex.drop(columns=['Index of Net Performance'], inplace=True)
ltcm_ex.head()


,Fund Capital ($billions),Gross Monthly Performancea,Net Monthly Performanceb,SPY (excess)
1994-03-31,1.0970,-0.0140,-0.0160,-0.0449
1994-04-30,1.0967,0.0107,0.0047,0.0079
1994-05-31,1.1964,0.0644,0.0494,0.0123
1994-06-30,1.1965,-0.0425,-0.0326,-0.0264
1994-07-31,1.3963,0.1123,0.0803,0.0287


In [117]:
ltcm_ex.columns = ['Fund Capital ($billions)', 'Gross Performance (excess)', 'Net Performance (excess)', 'SPY (excess)']

ltcm_ex.head()

,Fund Capital ($billions),Gross Performance (excess),Net Performance (excess),SPY (excess)
1994-03-31,1.0970,-0.0140,-0.0160,-0.0449
1994-04-30,1.0967,0.0107,0.0047,0.0079
1994-05-31,1.1964,0.0644,0.0494,0.0123
1994-06-30,1.1965,-0.0425,-0.0326,-0.0264
1994-07-31,1.3963,0.1123,0.0803,0.0287


# 2.1(a)

In [118]:
mean_vol_sharpe(ltcm_ex)[1:]

,mean,volatility,sharpe_ratio
Gross Performance (excess),0.2421,0.1362,1.7769
Net Performance (excess),0.1554,0.1118,1.3901
SPY (excess),0.1738,0.1123,1.5479


# 2.1(b)

In [119]:
risk_stats(ltcm_ex).iloc[:3,1:4].T

,Skewness,Excess Kurtosis,VaR (negated)
Gross Performance (excess),-0.2877,1.5866,0.0304
Net Performance (excess),-0.8102,2.9269,0.0264
SPY (excess),-0.4335,-0.362,0.0464


# 2.1 (c)

- The Sharpe Ratios are high compared to the SPY.
- Vols are similar to SPY
- The Risk Stats SKewness and Kurtosis are worse for the net series.

## 2.2 Market Regression
Using the series of net LTCM excess returns, denoted $\tilde{r} ^{LTCM}$
, estimate the following regression:
>$\tilde{r}^{LTCM}_{t} = \alpha + \beta^{m}\tilde{r}^{m}_{t}+\epsilon_{t}$

- Report $α$ and $β^m$. Report the $R^2$ stat.
- From this regression, does LTCM appear to be a “closet indexer”? <br>
- From the regression, does LTCM appear to deliver excess returns beyond the risk premium we expect from market exposure?

### 2.2(a)

In [120]:
regress(ltcm_ex['Net Performance (excess)'], ltcm_ex['SPY (excess)']) 

,Regression Parameters
const,0.1315
SPY (excess),0.1371
r-squared,0.0190


### 2.2(b) From this regression, does LTCM appear to be a “closet indexer”?

The closet indexer is a strategy that is similar to the market but not exactly the same. In this case, LTCM is not a closet indexer because the beta is not close to 1. Also, the alpha is not close to 0, that means that there is unexplainable return.

### 2.2(c) From the regression, does LTCM appear to deliver excess returns beyond the risk premium we expect from market exposure?

Yes, since the market is not able to explain LTCM returns (see regression above) we can conclude that LTCM is delivering excess returns not explained by the market. The alpha is this excess return and is positive. (13.15%)

### 2.3 Let’s check for non-linear market exposure. Run the following regression on LTCM’s net excess returns:
>$\tilde{r}^{LTCM}_{t} = \alpha + \beta_{1}\tilde{r}^{m}_{t}+ \beta_{2}(\tilde{r}^{m}_{t})^{2}+\epsilon_{t}$

- Report $β_1$, $β_2$, and the $R^2$ stat
- Does the quadratic market factor do much to increase the overall LTCM variation explained by the market?
- From the regression evidence, does LTCM’s market exposure behave as if it is long market options or short market options?
- Should we describe LTCM as being positively or negatively exposed to market volatility?

#### 2.3(a) Report $β_1$, $β_2$, and the $R^2$ stat

In [121]:
ltcm_ex['SPY (excess) squared'] = ltcm_ex['SPY (excess)']**2

regress(ltcm_ex['Net Performance (excess)'], ltcm_ex[['SPY (excess)', 'SPY (excess) squared']])

,Regression Parameters
const,0.1550
SPY (excess),0.1669
SPY (excess) squared,-1.9267
r-squared,0.0243


#### 2.3(b) Does the quadratic market factor do much to increase the overall LTCM variation explained by the market?

- The Quadratic Market Factor does not increase the overall LTCM variation explained by the market.
- The squared of SPY is not a good predictor of LTCM returns.

#### 2.3(c) From the regression evidence, does LTCM’s market exposure behave as if it is long market options or short market options?

- LTCM's market exposure is short market options.
- The quadratic or square factor is the representative of market options.

#### 2.3(d) Should we describe LTCM as being positively or negatively exposed to market volatility?

- LTCM is negatively exposed to market volatility. This is because the squared term is negative.
- Large movements in the market are bad for LTCM.

## 2.4 Asymmetric Regression
Let’s try to pinpoint the nature of LTCM’s nonlinear exposure. Does it come more from exposure to up-markets or down-markets? Run the following regression on LTCM’s net excess returns:
>$\tilde{r}^{LTCM}_{t} = \alpha + \beta\tilde{r}^{m}_{t}+ \beta_{u}max(\tilde{r}^{m}_{t}-k_{1}, 0)+ \beta_{d}max(k_{2}-\tilde{r}^{m}_{t}, 0)+\epsilon_{t}$ <br> <br>
where $k1 = .03$ and $k2 = −.03$. (This is roughly one standard deviation of $\tilde{r}^m$ .)

### 2.4(a) Report $β$, $β_u$, $β_d$, and the $R^2$ stat.

In [122]:
k1 = .03
k2 = -.03

ltcm_ex['Up'] = (ltcm_ex['SPY (excess)'] - k1).clip(0)
ltcm_ex['Down'] = (k2 - ltcm_ex['SPY (excess)']).clip(0)

regress(ltcm_ex['Net Performance (excess)'], ltcm_ex[['SPY (excess)','Up','Down']])

,Regression Parameters
const,0.1012
SPY (excess),0.4666
Up,-0.7821
Down,1.2896
r-squared,0.0555


### 2.4(b) Is LTCM long or short the call-like factor? And the put-like factor?

- $\beta^{Up}$ is negative so LTCM appears short the call-like factor.
- $\beta^{Down}$ is positive so LTCM appears long the put-like factor.

### 2.4 (c) Which factor moves LTCM more, the call-like factor, or the put-like factor?

The put-like factor moves LTCM more, as its absolute value is larger than the call-like factor.

### 2.4 (d) In the previous problem, you commented on whether LTCM is positively or negatively exposed to market volatility. Using this current regression, does this volatility exposure come more from being long the market’s upside? Short the market’s downside? Something else?

In the previous problem we commented that LTCM is negatively exposed to market vol. This volatility exposure must come more from being short the market's upside, as LTCM is short the call-like factor and long the put-like factor. 

## 3 The FX Carry Trade

Find an Excel data file, “fx carry data.xlsx”. The file has two sets of data:
- Risk-free rates across 5 currencies, as measured by annualized 3-month LIBOR rates.
- Spot FX rates, as direct quotes to the USD. (Note that all currencies are quoted as USD per the foreign currency.)

In [123]:
rf = pd.read_excel('fx_carry_data.xlsx',1, index_col=0)
log_rf = np.log(1 + rf)

log_rf.head()

,USD1M,GBP1M,EUR1M,CHF1M,JPY1M
DATE,,,,,
1999-01-31,0.0041,0.0049,0.0026,0.0010,3.3458e-04
1999-02-28,0.0041,0.0046,0.0026,0.0010,2.3226e-04
1999-03-31,0.0041,0.0044,0.0025,0.0010,1.4270e-04
1999-04-30,0.0041,0.0044,0.0021,0.0008,9.8953e-05
1999-05-31,0.0041,0.0044,0.0021,0.0008,7.4997e-05


In [124]:
fx = pd.read_excel('fx_carry_data.xlsx', 2, index_col=0)

log_fx = np.log(fx)

log_fx.head()

,USUK,USEU,USSZ,USJP
DATE,,,,
1999-01-31,0.4982,0.1285,-0.3484,-4.7536
1999-02-28,0.4717,0.0949,-0.3712,-4.7766
1999-03-31,0.4787,0.0777,-0.3904,-4.7743
1999-04-30,0.4753,0.0549,-0.4225,-4.7827
1999-05-31,0.4713,0.0413,-0.4240,-4.7948


### 3.1. The Static Carry Trade
Define the log return of holding the foreign currency using log values of the risk-free rate and log values of the FX rates: <br>
$ r^{i}_{t+1} = s^{i}_{t+1} − s^{i}_{t} + r^{f,i}_{t,t+1} $<br><br>
Then the excess log return relative to USD, is expressed as <br>
$ r^{i}_{t+1} = s^{i}_{t+1} − s^{i}_{t} + r^{f,i}_{t,t+1} - r^{f,\$}_{t,t+1} $ <br><br>
For each foreign currency, $i$, calculate the excess log return series, $ \tilde{r}_{t+1}$. Report the following
stats, (based on the excess log returns.) Annualize them.

- mean
- volatility
- Sharpe ratio <br>
What differences do you see across currencies?

In [125]:
log_rf_ex = log_rf.subtract(log_rf['USD1M'], axis=0)[log_rf.columns[1:]]

log_rf_ex_col = log_rf_ex.copy()

log_rf_ex_col.columns = log_fx.columns

ret = (log_rf_ex_col.shift() + log_fx.diff()).dropna()

ret.head()

,USUK,USEU,USSZ,USJP
DATE,,,,
1999-02-28,-0.0257,-0.0351,-0.0259,-0.0268
1999-03-31,0.0075,-0.0187,-0.0222,-0.0016
1999-04-30,-0.0031,-0.0245,-0.0352,-0.0124
1999-05-31,-0.0037,-0.0155,-0.0048,-0.0160
1999-06-30,-0.0157,-0.0128,-0.0211,-0.0045


In [126]:
mean_vol_sharpe(ret)

,mean,volatility,sharpe_ratio
USUK,-0.0035,0.0863,-0.0406
USEU,-0.0044,0.0947,-0.0459
USSZ,0.0043,0.0988,0.0437
USJP,-0.0174,0.0915,-0.1903


### What differences do you see across currencies?

THe VOLs are similar across currencies. THe USD/JPY FX pair has the lowest sharpe ratio. The means are very low except for the USD/JPY FX pair, which has a -1.7% mean.

### 3.2 Implications for UIP:

#### 3.2 (a) Do any of these stats contradict the (log version) of Uncovered Interest Parity (UIP)?

- The above results show that the mean return across FX pairs are not zero, though they are very small and could be considered zero. This contradicts the UIP.
- USD and JPY pair show a mean more than 1%, maybe indicative of market inefficiencies.

### 3.2(b) A long position in which foreign currency offered the best Sharpe ratio over the sample?

- THe only curency which offers a positive Shapre ratio for being long is USD/CHF.

### 3.2(c) Are there any foreign currencies for which a long position earned a negative excess return (in USD) over the sample?

- JPY, EUR and GBP all have negative excess returns.

## 3.3 Predicting FX
For each foreign currency, test whether interest-rate differentials can predict growth in the foreign-exchange rate. Do this by estimating the following forecasting regression: <br><br>
> $s^{i}_{t+1}-s^{i}_{t}=\alpha^{i}+\beta^{i}(r^{f,\$}_{t,t+1}-r^{f,i}_{t,t+1})+\epsilon^{i}_{t+1}$ <br> <br>
where $r^{f,i}$ denotes the risk-free rate of currency $i$, and $s^i$ denotes the FX rate for currency $i$.
Again, note that both $r^{f,\$}_{t,t+1}$ and $s_t$ are determined at time $t$.


#### 3.3(a) Make a table with columns corresponding to a different currency regression. Report the regression estimates $α_i$ and $β_i$ in the first two rows. Report the $R^2$ stat in the third row.

In [127]:
ys = log_fx.diff().dropna()
Xs = -log_rf_ex.shift().dropna()

fx_reg_pred = pd.DataFrame(data = None, index = [r'alpha',r'beta',r'r-squared'])

for i in range(0,len(ys.columns)):
    fx_reg_pred[ys.columns[i]] = regress(ys[ys.columns[i]],Xs[Xs.columns[i]])['Regression Parameters'].values
    
fx_reg_pred

,USUK,USEU,USSZ,USJP
alpha,-0.0059,0.0070,0.0436,-0.0060
beta,0.4858,-1.2564,-1.6466,0.3715
r-squared,0.0004,0.0026,0.0039,0.0005


#### 3.3(b) Suppose the foreign risk-free rate increases relative to the US rate.
#### i. For which foreign currencies would we predict a relative strengthening of the USD in the following period?
Ans:- If the foreign risk-free rate increases relative to the US rate, we would predict a relative strengthening of the USD against GBP and JPY. This is beacuse the regression coefficients for these currencies are positive. Hence if  $r^{f,\$}_{t,t+1}-r^{f,i}_{t,t+1}$  is negative, with a positive regression coefficient, we would predict a relative strengthening of the USD, i.e., $s^{i}_{t+1}-s^{i}_{t}$ is positive.


### ii. For which currencies would we predict relative weakening of the USD in the following period?

Ans: If the foreign risk-free rate increases relative to the US rate, we would predict a relative weakening of the USD against EUR and CHF. This is beacuse the regression coefficients for these currencies are negative. Hence if  $r^{f,\$}_{t,t+1}-r^{f,i}_{t,t+1}$  is negative, with a negative regression coefficient, we would predict a relative weakening of the USD, i.e., $s^{i}_{t+1}-s^{i}_{t}$ is negative.

### iii. This FX predictability is strongest in the case of which foreign currency?

The FX predictability is the strongest in the case of EUR currency. It has a very high regression coefficient of -1.2. Also, the alpha (unexplained part of the regression) is very low.

## 3.4. The Dynamic Carry Trade
Use this to write $\mathbb{E}_t [\tilde{r}^{i}_{t+1}]$ as a function of the interest-rate differential as well as $α$ and $β$ from this FX regression. <br>
$\mathbb{E}_t [s_{t+1} - s_{t}] = \alpha + \beta(r^{f,\$}_{t,t+1}-r^{f,i}_{t,t+1})$ <br><br>
Then use the definition of excess (log) returns on FX: <br>
$\tilde{r}^{i}_{t+1} = s_{t+1} - s_{t} - (r^{f,\$}_{t,t+1} - r^{f,i}_{t,t+1})$ <br><br>
Rearranging, this implies the following forecast for excess log returns: <br>
$\mathbb{E}[\tilde{r}^{i}_{t+1}] = \alpha + (\beta - 1)(r^{f,\$}_{t,t+1}-r^{f,i}_{t,t+1})$ <br><br>

### 3.4(a) Use your regression estimates from Problem 3 along with the formula above to calculate the fraction of months for which the estimated FX risk premium positive. That is, for each i, calculate how often in the time-series we have
$\mathbb{E}[\tilde{r}^{i}_{t+1}] > 0$.

In [128]:
alphas = (fx_reg_pred.loc[r'alpha'] / 12)
betas = fx_reg_pred.loc[r'beta']
Xs.columns = fx_reg_pred.columns

ret_ex_forecast = alphas.values + ((betas - 1) * Xs)

ret_ex_forecast.head()

,USUK,USEU,USSZ,USJP
DATE,,,,
1999-02-28,-6.8394e-05,-0.0028,-0.0046,-0.0029
1999-03-31,-2.3875e-04,-0.0029,-0.0046,-0.0029
1999-04-30,-3.2543e-04,-0.0031,-0.0046,-0.0030
1999-05-31,-3.2588e-04,-0.0038,-0.0050,-0.0030
1999-06-30,-3.3511e-04,-0.0039,-0.0050,-0.0030


In [129]:
month_frac = pd.DataFrame(data = None, columns = ret_ex_forecast.columns, index = ['% of Months'])

for col in ret_ex_forecast.columns:
    month_frac[col] = (len(ret_ex_forecast[ret_ex_forecast[col] > 0])/len(ret_ex_forecast)) * 100

ret_ex_forecast

month_frac

,USUK,USEU,USSZ,USJP
% of Months,23.8095,50.1832,63.0037,0.0


### 3.4 (b) Which currencies most consistently have a positive FX risk premium? And for which currencies does the FX risk premium most often go negative?

GBP, EUR and CHF have positive FX risk premium 24%, 50% and 63% of the time respectively. JPY on the other hand has a negative FX risk premium all the time.

### 3.4(c) Explain how we could use these conditional risk premia to improve the static carry trade returns calculated in Problem 1.

Forecasts of excess returns can be used for construction of a dynamic trading position. We could time the magnitude and direction of the currency trade based on the betas and alphas of the regression. We could also use the conditional risk premia to improve the static carry trade returns calculated in Problem 1. This is all assuming that the in-sample results we see here are even applicable out of sample.